## 直接根据上下文 生成文档分析

In [8]:
# 打开并读取Markdown文件
with open('./test_data/output/10010119/auto/10010119.md', 'r', encoding='utf-8') as file:
    content = file.read()

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
        
client = OpenAI()

print()
prompt = f"""
请总结并分析下列年度财报：
{content}
"""

system_prompt = """

你是一个金融分析专家，你需要帮助用户进行金融数据分析与总结。

"""
response = client.chat.completions.create(
    # model="gpt-3.5-turbo", # Please reduce the length of the messages
    model="gpt-3.5-turbo-16k",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=1024,
)

In [7]:
print(response.choices[0].message.content)

根据深圳和而泰智能控制股份有限公司2024年第一季度报告的主要财务数据，我们可以得出以下分析和总结：

1. 营业收入：2024年第一季度，公司实现营业收入1,981,896,074.22元，同比增长20.51%。这表明公司的业务表现良好，市场需求稳定增长。

2. 净利润：归属于上市公司股东的净利润为96,572,467.09元，同比增长18.06%。公司通过降低成本和提高效率，实现了利润的持续稳定增长。

3. 现金流量净额：经营活动产生的现金流量净额为-148,665,457.78元，同比下滑316.36%。主要原因是支付货款及员工薪酬奖金导致的经营性现金流下滑。这可能是季度性因素导致的，不会影响公司的正常生产经营。

4. 资产负债表：总资产为10,307,122,147.87元，同比增长3.44%。归属于上市公司股东的所有者权益为4,636,247,470.74元，同比增长2.04%。公司的资产规模和股东权益保持稳定增长。

综上所述，深圳和而泰智能控制股份有限公司在2024年第一季度取得了良好的业绩表现。公司的营业收入和净利润均实现了同比增长，资产规模和股东权益也保持了稳定增长。尽管经营性现金流净额下滑，但这可能是季度性因素导致的。公司的财务状况良好，具有稳定的发展前景。


## 利用大模型从contxt中进行实体抽取

In [13]:
# 读取context

# 打开并读取Markdown文件
with open('./test_data/output/10010119/auto/10010119.md', 'r', encoding='utf-8') as file:
    content = file.read()               

# 输出文件内容
print(len(content)) 

27655


In [16]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
        
client = OpenAI()

json = {
    "company_name":"公司名称",
    "stock_code":"股票代码",
    "report_time":"报表时间",
    "report_type":"报表类型",
    
    "main_industry":{
        
    "所属行业分类":"",
    "所属行业分类描述":"",
    "行业基本情况":"",
    "公司所处行业的发展阶段":"",
    "公司的行业地位":"",
    "新公布的法律、行政法规、部门规章、行业政策对所处行业的重大影响":""

    },
    
    "main_business":
    {
    "主营业务":"",
    "主要产品":"",
    "主要服务":""
    },
    "core_competitiveness":"核心竞争力",
    "business_analysis":"主营业务分析",

    "友商和上下游情况":
    {
    "友商":"",
    "上游企业":"",
    "下游":""
    },
    
    
    "financial_data":"财务数据",
    
    
    "研发投入情况":
    {
    "研发项目情况":"",
    "研发人员情况":"",
    "专利情况":""
    },
    
    "供应商和销售商":
    {
    "销售客户集中度情况":"",
    "供应商集中度情况":""
    },
    "公司发展战略":""
    
}
prompt = f"""

请对以下文本内容进行分析总结，然后完成json格式内容的填写,部分信息包含描述，填写的时候请自动替换内容，没有内容的请根据文本内容进行填充。
文本内容如下：{content}

输出json格式如下：
{json}

    
"""



system_prompt = """

你是一个实体抽取模型，你的任务是从多个段落中抽取出文本中的实体以及内容，并返回一个json格式的结果。
请保证内容的真实可靠，禁止胡编乱造，若未识别到该内容的，则暂时填写为null，
"""



# 128,000
#  27,655
# 支持结构化输出的最新快照gpt-4o目前指向此版本。

response = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
)



In [31]:
data = response.choices[0].message.content[7:-3]
print(data)


{
    "company_name": "深圳和而泰智能控制股份有限公司",
    "stock_code": null,
    "report_time": "2024年第一季度",
    "report_type": "季度报告",
    "main_industry": {
        "所属行业分类": null,
        "所属行业分类描述": null,
        "行业基本情况": "公司在全球20多个国家和地区进行了研发、运营中心及生产基地的布局，形成了贴近客户服务的机制，实现了客户需求快速响应。",
        "公司所处行业的发展阶段": "公司在产业全球化的大背景下，重点加大新领域的研发投入，深化海外市场布局。",
        "公司的行业地位": "作为行业全球化程度领先的企业，公司在海外市场拓展上占据有利竞争地位。",
        "新公布的法律、行政法规、部门规章、行业政策对所处行业的重大影响": null
    },
    "main_business": {
        "主营业务": "智能控制系统的研发、生产和销售。",
        "主要产品": "控制器业务板块产品。",
        "主要服务": "提供控制系统解决方案。"
    },
    "core_competitiveness": "公司通过持续进行供应链优化、研发设计优化等方式降本增效，严格控制各项成本开支，加强费用管理，提升生产及管理效率。",
    "business_analysis": "报告期内，公司整体经营情况持续向好，虽然受春节假期影响，但公司克服节后用工困难，积极组织生产保证交付，使得报告期内经营业绩持续增长。",
    "友商和上下游情况": {
        "友商": null,
        "上游企业": null,
        "下游": null
    },
    "financial_data": "2024年第一季度，公司实现营业收入1,981,896,074.22元，同比增长20.51%；归属于上市公司股东的净利润为96,572,467.09元，同比增长18.06%。",
    "研发投入情况": {
        "研发项目情况": "公司战略

In [35]:
import json

def write_string_to_json(string_data, file_path):
    """
    将字符串以字典的格式写入 JSON 文件中。

    :param string_data: 要写入的字符串
    :param file_path: 目标文件路径
    """
    try:
        # 将字符串解析为字典
        data = json.loads(string_data)
        
        # 将字典写入 JSON 文件
        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        
        print(f"字符串已成功写入 JSON 文件: {file_path}")
    except json.JSONDecodeError as e:
        print(f"字符串解析为 JSON 时发生错误: {e}")
    except Exception as e:
        print(f"写入 JSON 文件时发生错误: {e}")

        
write_string_to_json(data, 'result.json')

字符串已成功写入 JSON 文件: result.json
